# Optimizing Crop Rotation for Small-scale farmers in Rwanda

In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
data = pd.read_csv('soil.impact.csv')
data.head()

,Name,Fertility,Temperature,Rainfall,pH,Light_Hours,Light_Intensity,Rh,Nitrogen,Phosphorus,Potassium,Yield,Soil_Type,Season,Impact
0,Strawberry,Moderate,20.887923,747.860765,6.571548,13.091483,533.762876,91.197196,170.800381,118.670058,243.331211,20.369555,Loam,Summer,depleting
1,Strawberry,Moderate,18.062721,711.104329,6.251806,13.063016,505.789101,91.939623,179.290364,121.020244,246.910378,20.402751,Loam,Spring,depleting
2,Strawberry,Moderate,16.776782,774.038247,6.346916,12.945927,512.985617,91.387286,181.440732,116.936806,242.699601,19.158847,Loam,Summer,depleting
3,Strawberry,Moderate,14.281000,665.633506,6.259598,13.318922,484.860067,91.254598,176.165282,122.233153,237.096892,20.265745,Loam,Summer,depleting
4,Strawberry,Moderate,21.444490,806.531455,6.384368,13.312915,512.747307,92.354829,182.935334,126.088234,243.880364,20.397336,Loam,Spring,depleting


# Crop recommendation part

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from lime.lime_tabular import LimeTabularExplainer
from skimpy import skim
import shap

In [ ]:
# dataset
file_path = '1..soil.csv'
df = pd.read_csv(file_path)

#summary of the dataset using skimpy
skim(df)

In [ ]:
# missing values
print("\nMissing values:\n", df.isnull().sum())

# summary statistics for numerical features
print("\nSummary statistics:\n", df.describe())

# distribution of numerical features
plt.figure(figsize=(15, 10))
df.hist(bins=30, color='lightgreen', edgecolor='black')
plt.tight_layout()
plt.show()

# count plots for categorical features
categorical_columns = ['Fertility', 'Soil_Type', 'Season']
plt.figure(figsize=(15, 10))
for i, col in enumerate(categorical_columns, 1):
    plt.subplot(2, 3, i)
    sns.countplot(x=col, data=df, palette='Greens')
    plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Data Preprocessing

In [ ]:
# Data preprocessing
label_encoders = {col: LabelEncoder() for col in categorical_columns}

for col in categorical_columns:
    df[col] = label_encoders[col].fit_transform(df[col])

# encode the target variable (crop)
target_encoder = LabelEncoder()
y = target_encoder.fit_transform(df['Name'])

# var for training
X = df.drop(columns=['Name'])

# Standardize numerical columns
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Correlation Analysis

In [ ]:
numerical_columns = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numerical_columns].corr()

# heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='Greens', fmt='.2f', linewidths=0.5)
plt.title("Correlation Heatmap of Numerical Features")
plt.tight_layout()
plt.show()

### Model Training and Evaluation

In [ ]:
# define 4 models and accuracy
models = {
    'Gradient Boosting': GradientBoostingClassifier(),
    'XGBoost': XGBClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=200)
}

# train and evaluate models
model_results = {}
feature_importances = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_results[name] = model.score(X_test, y_test)

    # feature importance for tree-based models
    if name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
        feature_importances[name] = model.feature_importances_
    elif name == 'Logistic Regression':
        feature_importances[name] = model.coef_[0]  # Use coefficients for Logistic Regression

# Print model performance
print("Model Performance:")
for model_name, accuracy in model_results.items():
    print(f"{model_name}: {accuracy:.4f}")

# feature importance- plots
for model_name, importance in feature_importances.items():
    plt.figure(figsize=(10, 6))
    sns.barplot(x=importance, y=X.columns)
    plt.title(f'Feature Importance - {model_name}')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.show()


In [ ]:
# Crop recommendation FINAL!!!!!!

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib  

# Load the dataset (assuming 'soil.csv' as the file name)
file_path = 'soil.impact.csv'
soil_data = pd.read_csv(file_path)

# Apply One-Hot Encoding to the 'Season' column (creates columns like 'Season_Spring', 'Season_Summer', etc.)
soil_data_encoded = pd.get_dummies(soil_data, columns=['Season'])

# Prepare the data for Random Forest Classifier
X = soil_data_encoded[['Temperature', 'Rainfall', 'Light_Intensity', 'Nitrogen', 'Phosphorus', 'Potassium'] + list(soil_data_encoded.filter(like='Season_').columns)]
y = soil_data_encoded['Name']

# Encode the target labels (crop names)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# FIT the model (this is the critical step to ensure the model is trained)
rf_classifier.fit(X_train, y_train)

# Save the trained RandomForest model and LabelEncoder using joblib
#joblib.dump(rf_classifier, 'random_forest.pkl')
#joblib.dump(label_encoder, 'label_encoder.pkl')

# Function to predict the top 3 suitable crops based on input features
def predict_top_crops(input_features, model, label_encoder, top_n=3):
    # Get probabilities for each class (crop)
    probabilities = model.predict_proba([input_features])[0]
    
    # Get the top_n crops based on probabilities
    top_indices = np.argsort(probabilities)[-top_n:][::-1]
    
    # Decode the labels back to crop names
    top_crops = label_encoder.inverse_transform(top_indices)
    
    return top_crops

# input for testing (e.g., temperature, rainfall, N, P, K, Spring = 1, others = 0)
example_input = [20.0, 750.0, 180.0, 180.0, 120.0, 240.0, 0, 0, 1, 0]  # Autum is active (1), others (Summer, Spring, Winter) are 0

# Predict the top 3 crops using the trained model
top_crops = predict_top_crops(example_input, rf_classifier, label_encoder)

# Print the top 3 crops in a formatted manner
print(f"The top 3 crops suitable for actual conditions are: {top_crops[0]}, {top_crops[1]}, and {top_crops[2]}.")


In [ ]:
# Filter dataset 
sandy_soil_crops = data[data['Soil_Type'] == 'Sandy']['Name'].unique()
sandy_soil_crops    # array(['Green Peas']

winter_crops = data[data['Season'] == 'Winter']['Name'].unique()
winter_crops   # array(['Kale', 'Cauliflowers', 'Broccoli']

 # Crop rotation with Genetic Algorithms 

### Feature engineering

In [ ]:
# first step for crop rotation- calculate NPK means and categorize into 3 types: restorative,neutral, depleting-- creating a new column: Impact to work with 

# Calculate mean NPK values
mean_nitrogen = soil_data['Nitrogen'].mean()
mean_phosphorus = soil_data['Phosphorus'].mean()
mean_potassium = soil_data['Potassium'].mean()

#classify each crop based on NPK values
def classify_impact(row):
    conditions = [
        (row['Nitrogen'] < mean_nitrogen) and (row['Phosphorus'] < mean_phosphorus) and (row['Potassium'] < mean_potassium),
        (row['Nitrogen'] > mean_nitrogen) and (row['Phosphorus'] > mean_phosphorus) and (row['Potassium'] > mean_potassium)
    ]
    choices = ['restorative', 'depleting']
    return np.select(conditions, choices, default='neutral')

# create the new column
#soil_data['Impact'] = soil_data.apply(classify_impact, axis=1)


In [ ]:
# crop rotation FINAL!!!!!!!!
import random
from deap import base, creator, tools, algorithms
import pandas as pd

# dataset
file_path = 'soil.impact.csv'
data = pd.read_csv(file_path)


impact_scores = {'restorative': 1, 'neutral': 0, 'depleting': -1}
data['Impact_Score'] = data['Impact'].map(impact_scores)

crops = data['Name'].unique()  
impact_data = data[['Name', 'Impact']].drop_duplicates() 

# genetic algorithm setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize fitness (yield, impact)
creator.create("Individual", list, fitness=creator.FitnessMax)

# create individual (crop rotation plan for 3 years with 3 crops per year = 9 results)
def create_individual():
    return [random.choice(impact_data.values.tolist()) for _ in range(9)]  

# Fitness function for evaluating each crop rotation plan
def evaluate(individual):
    unique_crops = len(set(crop for crop, _ in individual))  # reward for crop diversity
    random_yield = sum(random.uniform(10, 30) for _ in individual)  # random yield because actual yield doesn't give good results
    
    # evaluation based on impact categories
    impact_score = 0
    for _, impact in individual:
        if impact == "restorative":
            impact_score += 1  # 1 point for restorative crops
        elif impact == "neutral":
            impact_score += 0  # neutral crops get 0 points
        elif impact == "depleting":
            impact_score -= 1  # penalize depleting crops -1

    return unique_crops + random_yield + impact_score,  # Total fitness value

# GA components
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic operators
toolbox.register("mate", tools.cxTwoPoint)  # two-point crossover
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.4)  # shuffle mutation
toolbox.register("select", tools.selTournament, tournsize=3)  # tournament selection
toolbox.register("evaluate", evaluate)

# Genetic Algorithm parameters
population_size = 8000  # population size
generations = 30  #  generations
mutation_prob = 0.1 # probability of mutation
crossover_prob = 0.5 # probability of crossover

# Generate the initial population
population = toolbox.population(n=population_size)

# Run the genetic algorithm
result_population, log = algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=generations, verbose=False)

# Get the best individual (crop rotation plan)
best_individual = tools.selBest(result_population, 1)[0]
result= pd.DataFrame(best_individual)
print("Best Crop Rotation Plan: ", result)
